# Imports

In [ ]:
pip install transformers

In [ ]:
pip install textstat


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.0 MB/s eta 0:00:00


In [ ]:
pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 1.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd

# Load the data

In [ ]:
pd.set_option('max_colwidth', 400)

df = pd.read_csv('annotated_trees_101.csv', index_col=0)

In [ ]:
df.columns

Index(['node_id', 'tree_id', 'timestamp', 'author', 'text', 'parent',
       'Aggressive', 'AgreeBut', 'AgreeToDisagree', 'Alternative', 'Answer',
       'AttackValidity', 'BAD', 'Clarification', 'Complaint', 'Convergence',
       'CounterArgument', 'CriticalQuestion', 'DirectNo', 'DoubleVoicing',
       'Extension', 'Irrelevance', 'Moderation', 'NegTransformation',
       'Nitpicking', 'NoReasonDisagreement', 'Personal', 'Positive',
       'Repetition', 'RephraseAttack', 'RequestClarification', 'Ridicule',
       'Sarcasm', 'Softening', 'Sources', 'ViableTransformation',
       'WQualifiers'],
      dtype='object')

In [ ]:
df['Irrelevance'].value_counts()

0    9984
1     575
Name: Irrelevance, dtype: int64

In [ ]:
result = df.groupby('tree_id')['Irrelevance'].sum().reset_index()
result = result.sort_values(by='Irrelevance', ascending=False)
trees = list(result.tree_id)

# Pre-processing

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text, remove_stopwords=True, use_stemming=False, use_lemmatization=True):
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Convert to lowercase
    text = text.lower()

    # Tokenization (split the text into words)
    words = text.split()

    # Remove stopwords
    if remove_stopwords:
        stop_words = set(stopwords.words('english'))
        words = [word for word in words if word not in stop_words]

    # Apply stemming
    if use_stemming:
        stemmer = PorterStemmer()
        words = [stemmer.stem(word) for word in words]

    # Apply lemmatization
    if use_lemmatization:
        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word) for word in words]

    # Join the processed words back into a single string
    processed_text = ' '.join(words)

    return processed_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


# Main Method

## Functions

In [ ]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained BERT model for sentence embeddings
model = SentenceTransformer("stsb-roberta-large")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
import numpy as np
from math import sqrt, pow, exp
import torch

# Define a function to calculate semantic similarity
def calculate_semantic_similarity(text1, text2):
    embeddings1 = model.encode(text1, convert_to_tensor=True)
    embeddings2 = model.encode(text2, convert_to_tensor=True)

    cosine_similarity = util.cos_sim(embeddings1, embeddings2)[0][0].item()
    euclidean_distance = calculate_semantic_eucl(embeddings1, embeddings2)
    return cosine_similarity, euclidean_distance

def calculate_semantic_cosine(embeddings1, embeddings2):
    cosine_similarity = util.cos_sim(embeddings1, embeddings2)[0][0].item()
    return cosine_similarity

def calculate_semantic_eucl(embeddings1, embeddings2):
    euclidean_distance = torch.sqrt(torch.sum((embeddings1 - embeddings2) ** 2)).item()
    return euclidean_distance


# Calculate mean and standard deviation
def get_stat(lst):
  mean_value = np.mean(lst)
  std_deviation = np.std(lst)
  return mean_value, std_deviation

In [ ]:
import textstat

def calculate_readability_score(text):
    try:
        # Calculate Flesch-Kincaid readability score
        readability_score = textstat.flesch_kincaid_grade(text)
        return readability_score
    except Exception as e:
        print(f"Error calculating readability score: {e}")
        return 0



In [ ]:
import matplotlib.pyplot as plt

def plot_curr(texts_irrel,red,count,all_score,name):
  red = [all_txt.index(item) for item in texts_irrel]
  plt.figure(figsize=(15, 6))  # Adjust the width and height as needed
  # Plot the line plot
  plt.plot(count, all_score, marker='o', linestyle='-')
  plt.scatter([count[i] for i in red], [all_score[i] for i in red], color='red', zorder=5)

  plt.xlabel('Comment depth')
  plt.ylabel('Cosine similarity score')
  plt.title(f'Change in Cosine similarity score between the root and the comment based on comment depth - {name}')
  plt.grid(True)
  plt.show()


## Root only

In [ ]:
res_df = pd.DataFrame()
res_lst = []
for tree in trees:
  try:
    df_chose = df[df['tree_id'] == tree]
    df_chose = df_chose[df_chose['text'].apply(lambda x: len(x.split()) >= 7)]
    # process root
    root = df_chose[df_chose['parent'] == -1].text
    root = root.reset_index()
    root_txt = list(root['text'])[0]
    root_txt_clean = preprocess_text(root_txt)
    root_txt_clean = model.encode(root_txt_clean, convert_to_tensor=True)

    # create groups
    all_txt = list(df_chose['text'])[1:]

    irrelevent = df_chose[df_chose['Irrelevance'] == 1].text
    irrelevent = irrelevent.reset_index()
    texts_irrel = list(irrelevent['text'])

    rrelevent = df_chose[df_chose['Irrelevance'] == 0].text
    rrelevent = rrelevent.reset_index()
    texts_relev = list(rrelevent['text'])

    # run the main loop
    irre_score_cos = []
    rel_score_cos = []
    all_score_cos = []

    irre_score_euc = []
    rel_score_euc = []
    all_score_euc = []
    # red  = []
    # count_ = 0
    # count = [i for i in range(len(all_txt))]
    for txt in all_txt:
      # if txt in texts_irrel:
      #   red.append(count_)
      txt_prcoess = preprocess_text(txt)
      txt_prcoess = model.encode(txt_prcoess, convert_to_tensor=True)

      score_cos = calculate_semantic_cosine(txt_prcoess,root_txt_clean)
      score_euc = calculate_semantic_eucl(txt_prcoess,root_txt_clean)

      print(score_cos, score_euc)
      if txt in texts_irrel:
        irre_score_cos.append(score_cos)
        irre_score_euc.append(score_euc)
      else:
        rel_score_cos.append(score_cos)
        rel_score_euc.append(score_euc)

      all_score_cos.append(score_cos)
      all_score_euc.append(score_euc)

    # get stats and plot the results
    c1, _ = get_stat(rel_score_cos)
    c2, _ = get_stat(irre_score_cos)
    c_total, _ = get_stat(all_score_cos)

    e1, _ = get_stat(rel_score_euc)
    e2, _ = get_stat(irre_score_euc)
    e_total, _ = get_stat(all_score_euc)


    len_irev = len(texts_irrel)
    len_relv = len(texts_relev)
    total = len(all_txt)
    res_lst.append((tree,total,len_relv,len_irev,c_total,c1,c2,e_total,e1,e2))
    # plot_curr(texts_irrel,red,count,all_score,tree)
  except Exception as e:
    print(f'Error in {tree} - {e}')

## Weighted sliding window

In [ ]:
# use weights

# if k is the next after root then root is 1.0

# if k is the second index then root = 0.7 and k-1 = 0.3

# if k is the third index then root = 0.6, k-1 = 0.25 and k-2 = 0.15

# else = 0.5 for root, 0.25 for k-1, 0.15 for k-2 and 0.1 for k-3

res_lst = []
for tree in trees:
  try:
    df_chose = df[df['tree_id'] == tree]
    df_chose = df_chose[df_chose['text'].apply(lambda x: len(x.split()) >= 7)]
    # process root
    root = df_chose[df_chose['parent'] == -1].text
    root = root.reset_index()
    root_txt = list(root['text'])[0]
    root_txt_clean = preprocess_text(root_txt)
    root_txt_clean = model.encode(root_txt_clean, convert_to_tensor=True)

    # create groups
    all_txt = list(df_chose['text'])[1:]

    irrelevent = df_chose[df_chose['Irrelevance'] == 1].text
    irrelevent = irrelevent.reset_index()
    texts_irrel = list(irrelevent['text'])

    rrelevent = df_chose[df_chose['Irrelevance'] == 0].text
    rrelevent = rrelevent.reset_index()
    texts_relev = list(rrelevent['text'])

    # run the main loop
    irre_score = []
    rel_score = []
    all_score = []

    irre_score_euc = []
    rel_score_euc = []
    all_score_euc = []


    embed_text = []
    for txt in all_txt:
      proccesed = preprocess_text(txt)
      embed_text.append(model.encode(proccesed, convert_to_tensor=True))

    for i in range(len(all_txt)):
      txt_embed = embed_text[i]
      txt = all_txt[i]


      # cosine only
      root_score = calculate_semantic_cosine(txt_embed,root_txt_clean)
      if i == 0:
        total_score = root_score
      elif i == 1:
        prev_txt = embed_text[0]
        prev_score = calculate_semantic_cosine(txt_embed,prev_txt)
        total_score = 0.6 * root_score + 0.4 * prev_score
      elif i == 2:
        prev_txt_0 = embed_text[0]
        prev_txt_1 = embed_text[1]
        prev_score_0 = calculate_semantic_cosine(txt_embed,prev_txt_0)
        prev_score_1 = calculate_semantic_cosine(txt_embed,prev_txt_1)
        total_score = 0.5 * root_score + 0.35 * prev_score_0 + 0.15 * prev_score_1
      else:
        prev_txt_0 = embed_text[i-1]
        prev_txt_1 = embed_text[i-2]
        prev_txt_2 = embed_text[i-3]

        prev_score_0 = calculate_semantic_cosine(txt_embed,prev_txt_0)
        prev_score_1 = calculate_semantic_cosine(txt_embed,prev_txt_1)
        prev_score_2 = calculate_semantic_cosine(txt_embed,prev_txt_2)
        total_score = 0.4 * root_score + 0.3 * prev_score_0 + 0.2 * prev_score_1 + 0.1 * prev_score_2

      # Euclidean only
      root_score_euc = calculate_semantic_eucl(txt_embed,root_txt_clean)
      if i == 0:
        total_score_euc = root_score_euc
      elif i == 1:
        prev_txt = embed_text[0]
        prev_score = calculate_semantic_eucl(txt_embed,prev_txt)
        total_score_euc = 0.6 * root_score_euc + 0.4 * prev_score
      elif i == 2:
        prev_txt_0 = embed_text[0]
        prev_txt_1 = embed_text[1]
        prev_score_0 = calculate_semantic_eucl(txt_embed,prev_txt_0)
        prev_score_1 = calculate_semantic_eucl(txt_embed,prev_txt_1)
        total_score_euc = 0.5 * root_score_euc + 0.35 * prev_score_0 + 0.15 * prev_score_1
      else:
        prev_txt_0 = embed_text[i-1]
        prev_txt_1 = embed_text[i-2]
        prev_txt_2 = embed_text[i-3]

        prev_score_0 = calculate_semantic_eucl(txt_embed,prev_txt_0)
        prev_score_1 = calculate_semantic_eucl(txt_embed,prev_txt_1)
        prev_score_2 = calculate_semantic_eucl(txt_embed,prev_txt_2)
        total_score_euc = 0.4 * root_score_euc + 0.3 * prev_score_0 + 0.2 * prev_score_1 + 0.1 * prev_score_2

      print(total_score, total_score_euc)
      if txt in texts_irrel:
        irre_score_cos.append(total_score)
        irre_score_euc.append(total_score_euc)
      else:
        rel_score_cos.append(total_score)
        rel_score_euc.append(total_score_euc)

      all_score_cos.append(total_score)
      all_score_euc.append(total_score_euc)

    # get stats and plot the results
    c1, _ = get_stat(rel_score_cos)
    c2, _ = get_stat(irre_score_cos)
    c_total, _ = get_stat(all_score_cos)

    e1, _ = get_stat(rel_score_euc)
    e2, _ = get_stat(irre_score_euc)
    e_total, _ = get_stat(all_score_euc)


    len_irev = len(texts_irrel)
    len_relv = len(texts_relev)
    total = len(all_txt)
    res_lst.append((tree,total,len_relv,len_irev,c_total,c1,c2,e_total,e1,e2))
    # plot_curr(texts_irrel,red,count,all_score,tree)
  except Exception as e:
    print(f'Error in {tree} - {e}')

Streaming output truncated to the last 5000 lines.
0.20548028424382211 0.027079210322787338
0.5457183808088303 0.038899525117481744
0.5091762512922287 0.0365890460497247
0.5302043654024602 0.03588687063135999
0.6076722145080566 0.0389161575529548
0.7000949382781982 0.043692461698634784
0.4483948290348053 0.03231552844315873
0.4510771155357361 0.03273153883073156
0.5167157411575318 0.034485699382069065
0.38282332569360733 0.030907679561098845
0.38639889359474183 0.03063465154083738
0.5731697887182235 0.037434089229699534
0.5748323559761048 0.03720827138336342
0.6909017324447633 0.04364424979531686
0.7155416905879973 0.04493330941204538
0.7580840170383455 0.05043002772153793
0.6663090825080872 0.04203143480978337
0.7271394968032836 0.04640377899507988
0.7050508618354798 0.04518872146214725
0.21421583890914916 0.02751555790721636
0.10156853273510932 0.026860648505184104
0.36340423226356505 0.03318981252050662
0.3726807594299316 0.031599460454757373
0.2825798079371452 0.029449134704745732


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


0.4657599925994873 0.03166551368461055
0.4807567834854126 0.03379142988714885
0.35831913799047466 0.029859865309304255
0.49696145057678226 0.03536523687227691
0.5855370342731475 0.04100500828276659
0.2485775351524353 0.028390456528040637
0.47972719669342034 0.03445048694589799
0.3446982264518738 0.030195940439697016
0.43898653686046596 0.032212401975092106
0.42011782526969915 0.03149888385415241
0.5409785509109497 0.03533291139561579
0.5692740440368652 0.03662658249244939
0.3964957773685455 0.031190118193084027
0.4191676259040833 0.032153619711435634
0.5213768601417542 0.03598753258675485
0.31051041781902317 0.029392127732978786
0.32926797568798066 0.029479361460953604
0.5342509180307389 0.034592114700925276
0.4614933460950852 0.03382882266249763
0.5694016098976136 0.03674017805768458
0.5000985264778137 0.033664679369287964
0.47487497031688686 0.03313123727846753
0.10159974396228791 0.024960251511969823
0.3640717945992947 0.03063988415763125
0.21190710663795473 0.027158876896665546
0.4

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


0.655171275138855 0.039977210257336214
0.6137572646141052 0.03837252937564061
0.7233542829751968 0.04625324674435619
0.6505115032196045 0.04105786809190766
0.5146547615528106 0.035315542129672764
0.5686672091484071 0.03664005114320429
0.45574083924293524 0.033972176038838835
0.4947525888681411 0.03392625033607558
0.4678988724946976 0.033475084214288256
0.4995142012834549 0.03445845944036943
0.5150031387805939 0.03438530738636695
0.523994755744934 0.03551917869993944
0.1868006616830826 0.027142795388152157
0.4172804281115532 0.03252680320000971
0.4619087159633637 0.03342212318515051
0.25510555207729335 0.02819215038029265
0.47774279117584234 0.03309315057872527
0.4917965233325958 0.03373284164655487
0.5295128643512725 0.03488391059614223
0.5208797872066497 0.03416957322667156
0.6209010839462281 0.03864469675528254
0.653177547454834 0.04106570280068022
0.5320700109004974 0.03509528056373965
0.6549431622028351 0.040620772416521664
0.7172022104263306 0.04437430179571089
0.6661025166511535 

...

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


0.42724353075027466 0.031245411116302576
0.4019877791404724 0.031874640681502295
0.520338848233223 0.03446258992254313
0.38193916380405424 0.030491569102331965
0.3366854816675186 0.0300538249107579
0.23649300038814547 0.02822242989920283
0.44748938083648687 0.03285973029220586
0.38864576220512387 0.03136560694027844
0.4997449487447739 0.034299422197706014
0.5799512147903443 0.036628228487845665
0.00271151326596737 0.0243476738899596
0.333021867275238 0.02963189022796249
0.4182150393724442 0.03279419605160166
0.41364098638296126 0.031999959623662824
0.53481425344944 0.03477682642237508
0.5173778384923935 0.03424239094410175
0.3779423356056214 0.030702917705640508
0.5295520007610321 0.03494911740898238
0.6368850886821747 0.042107311489598995
0.5732035130262375 0.036653371631991796
0.4225918680429458 0.030991446054635698
0.3394354254007339 0.030253096300133658
0.3652672797441482 0.029784176025374445
0.3260455965995789 0.029111442626818314
0.3986298426985741 0.03197724835716882
0.517431759

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


0.3151988387107849 0.02938562626053378
0.41697688102722164 0.03211990566171103
0.39517384469509126 0.03126259627559133
0.4329450875520706 0.03209750379857666
0.26020466387271884 0.028345157075576364
0.49688440859317784 0.0339273469669911
0.6283968329429627 0.04004521858767729
0.5631843566894531 0.03701176530092385
0.6281682401895523 0.03911241813136852
0.570615178346634 0.03630075968099692
0.6293206512928009 0.039998266317571994
0.40557352900505067 0.030978433633861976
0.5317772686481477 0.03516612515675873
0.5102715820074081 0.03417958730134347
0.46308151483535764 0.032358951425612584
0.5321902692317962 0.03489268011406767
0.4717711418867111 0.033803849906201916
0.4647497355937958 0.03262959183892128
0.3948894679546356 0.031116341082630273
0.37688651680946356 0.030214177435624046
0.3318141847848893 0.030057297545033355
0.09407161381095648 0.026036949811080082
0.36710375994443895 0.032068889534703236
0.5396051853895187 0.03921338416816146
0.5874967120587826 0.038571269893215035
0.41702

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
res_df = pd.DataFrame(res_lst,columns = ['Tree Name','N_comments','N_relevant','N_irrelevant','Total Cosine mean','Relevant Cosine mean','Irrelevant Cosine mean','Total Euclidean mean','Relevant Euclidean mean','Irrelevant Euclidean mean'])
res_df.fillna(0,inplace=True)
res_df['% Irrelevant'] = res_df['N_irrelevant'] / res_df['N_comments']
res_df = res_df.set_index('Tree Name')
res_df.to_csv('results_approach2.csv')

In [ ]:
res_df = pd.read_csv('results_approach2.csv')
res_df = res_df.iloc[:,1:]
res_df = res_df.set_index('Tree Name')

In [ ]:
res_df = res_df[res_df['N_irrelevant'] >3]
res_df.to_csv('result_new.csv')

## Create embeddings dataset

In [ ]:
# Create an embeddings and class dataframe
df_lst = []
for tree in trees:
  try:
    count = -1
    df_chose = df[df['tree_id'] == tree]
    df_chose = df_chose[df_chose['text'].apply(lambda x: len(x.split()) >= 7)]
    # process root
    root = df_chose[df_chose['parent'] == -1].text
    root = root.reset_index()
    root_txt = list(root['text'])[0]
    root_txt_clean = preprocess_text(root_txt)
    root_txt_clean = model.encode(root_txt_clean, convert_to_tensor=True)
    df_lst.append((tree,root_txt_clean,count,1))
    count += 1
    # create groups
    all_txt = list(df_chose['text'])[1:]

    irrelevent = df_chose[df_chose['Irrelevance'] == 1].text
    irrelevent = irrelevent.reset_index()
    texts_irrel = list(irrelevent['text'])

    rrelevent = df_chose[df_chose['Irrelevance'] == 0].text
    rrelevent = rrelevent.reset_index()
    texts_relev = list(rrelevent['text'])

    embed_text = []
    for txt in all_txt:
      proccesed = preprocess_text(txt)
      embedded_text = model.encode(proccesed, convert_to_tensor=True)
      if txt in texts_relev:
        cls = 0
      else:
        cls = 1


      df_lst.append((tree,embedded_text,count,cls))
      count +=1

  except:
    pass

total_df = pd.DataFrame(df_lst,columns = ['Tree Name','Embeddings','Comment_n','Class'])
total_df = total_df.set_index('Tree Name')
df['Embeddings'] = df['Embeddings'].apply(lambda x: x.numpy())

total_df.to_csv('Embedded.csv')

In [ ]:
total_df = pd.read_csv('Embedded.csv')

In [ ]:
total_df
total_df = total_df.set_index('Tree Name')
# Apply the conversion function to every row in the DataFrame
total_df['Embeddings'] = total_df['Embeddings'].apply(lambda x: torch.tensor(eval(x)))


ValueError: could not convert string to float: '...'

## K Sliding Window With root

In [ ]:
all_df = pd.DataFrame()

### K == 1

In [ ]:
weights = [(0.5,0.5), (0.25,0.75), (0.75,0.25)]
root_w = -1
full_house = []
col_names = ['Tree_Name','n','Class','W1-K1-R','W2-K1-R','W3-K1-R']


result = total_df.groupby('Tree Name')['Class'].sum().reset_index()
trees = list(result['Tree Name'])


for tree in trees:
  try:
    print(tree)
    df_chose = total_df[total_df.index == tree]
    order = list(df_chose['Comment_n'])[1:]
    cls =  list(df_chose['Class'])[1:]
    embeddings =  list(df_chose['Embeddings'])[1:]

    root = df_chose[df_chose['Comment_n'] == -1].Embeddings
    root = root.reset_index()
    root_txt = list(root['Embeddings'])[0]


    for i in order:
      cls_curr = cls[i]
      curr = embeddings[i]
      root_score = calculate_semantic_cosine(root_txt,curr)

      if i == 0:
        total_score = root_score
        full_house.append((tree,i,cls_curr,total_score,total_score,total_score))

      else:
        diff_w = []
        for w_set in weights:
          w1,w2 = w_set

          prev_txt_0 = embeddings[i-1]
          prev_score_0 = calculate_semantic_cosine(curr,prev_txt_0)
          total_score = w1 * root_score + w2 * prev_score_0 + w3
          diff_w.append(total_score)

        full_house.append((tree,i,cls_curr,diff_w[0],diff_w[1],diff_w[2]))

  except:
    print('Error')

full_house_df = pd.DataFrame(full_house,columns = col_names)
all_df = full_house_df

4r2a4d


TypeError: new(): invalid data type 'str'

### K == 2

In [ ]:
weights = [(0.333,0.333,0.333), (0.30,0.45,0.25), (0.6,0.3,0.1)]
full_house = []
col_names = ['Tree_Name','n','Class','W1-K2-R','W2-K2-R','W3-K2-R']

result = total_df.groupby('Tree Name')['Class'].sum().reset_index()
trees = list(result['Tree Name'])

for tree in trees:
  try:

    df_chose = total_df[total_df.index == tree]
    order = list(df_chose['Comment_n'])[1:]
    cls =  list(df_chose['Class'])[1:]
    embeddings =  list(df_chose['Embeddings'])[1:]


    # process root
    root = df_chose[df_chose['Comment_n'] == -1].Embeddings
    root = root.reset_index()
    root_txt = list(root['Embeddings'])[0]


    for i in order:
      cls_curr = cls[i]
      curr = embeddings[i]
      root_score = calculate_semantic_cosine(root_txt,curr)

      if i == 0:
        total_score = root_score
        full_house.append((tree,i,cls_curr,total_score,total_score,total_score))

      elif i == 1:
        prev_txt = embeddings[0]
        prev_score = calculate_semantic_cosine(curr,prev_txt)
        total_score = 0.6 * root_score + 0.4 * prev_score
        full_house.append((tree,i,cls_curr,total_score,total_score,total_score))

      else:
        diff_w = []
        for w_set in weights:
          w1,w2,w3 = w_set

          prev_txt_0 = embeddings[i-1]
          prev_txt_1 = embeddings[i-2]

          prev_score_0 = calculate_semantic_cosine(curr,prev_txt_0)
          prev_score_1 = calculate_semantic_cosine(curr,prev_txt_1)
          total_score = w1 * root_score + w2 * prev_score_0 + w3 * prev_score_1
          diff_w.append(total_score)

        full_house.append((tree,i,cls_curr,diff_w[0],diff_w[1],diff_w[2]))

  except:
    print('Error')

full_house_df = pd.DataFrame(full_house,columns = col_names)
all_df = all_df.merge(full_house_df.iloc[:, 3:], left_index=True, right_index=True)


### K == 3

In [ ]:

weights = [(0.25,0.25,0.25,0.25), (0.5,0.25,0.15,0.10), (0.4,0.3,0.2,0.1)],
full_house = []
col_names = ['Tree_Name','n','Class','W1-K3-R','W2-K3-R','W3-K3-R']

result = total_df.groupby('Tree Name')['Class'].sum().reset_index()
trees = list(result['Tree Name'])

for tree in trees:
  try:

    df_chose = total_df[total_df.index == tree]
    order = list(df_chose['Comment_n'])[1:]
    cls =  list(df_chose['Class'])[1:]
    embeddings =  list(df_chose['Embeddings'])[1:]

    root = df_chose[df_chose['Comment_n'] == -1].Embeddings
    root = root.reset_index()
    root_txt = list(root['Embeddings'])[0]


    for i in order:
      cls_curr = cls[i]
      curr = embeddings[i]
      root_score = calculate_semantic_cosine(root_txt,curr)

      if i == 0:
        total_score = root_score
        full_house.append((tree,i,cls_curr,total_score,total_score,total_score))

      elif i == 1:
        prev_txt = embeddings[0]
        prev_score = calculate_semantic_cosine(curr,prev_txt)
        total_score = 0.6 * root_score + 0.4 * prev_score
        full_house.append((tree,i,cls_curr,total_score,total_score,total_score))

      elif i == 2:
        prev_txt_0 = embeddings[0]
        prev_txt_1 = embeddings[1]
        prev_score_0 = calculate_semantic_cosine(curr,prev_txt_0)
        prev_score_1 = calculate_semantic_cosine(curr,prev_txt_1)
        total_score = 0.5 * root_score + 0.35 * prev_score_0 + 0.15 * prev_score_1
        full_house.append((tree,i,cls_curr,total_score,total_score,total_score))

      else:
        diff_w = []
        for w_set in weights:
          w1,w2,w3,w4 = w_set

          prev_txt_0 = embeddings[i-1]
          prev_txt_1 = embeddings[i-2]
          prev_txt_2 = embeddings[i-3]

          prev_score_0 = calculate_semantic_cosine(curr,prev_txt_0)
          prev_score_1 = calculate_semantic_cosine(curr,prev_txt_1)
          prev_score_2 = calculate_semantic_cosine(curr,prev_txt_2)
          total_score = w1 * root_score + w2 * prev_score_0 + w3 * prev_score_1 + w4 * prev_score_2
          diff_w.append(total_score)

        full_house.append((tree,i,cls_curr,diff_w[0],diff_w[1],diff_w[2]))

  except:
    print('Error')

full_house_df = pd.DataFrame(full_house,columns = col_names)
all_df = all_df.merge(full_house_df.iloc[:, 3:], left_index=True, right_index=True)


### K == 4

In [ ]:
weights = [(0.20,0.20,0.20,0.20,0.20), (0.4,0.20,0.15,0.15,0.10), (0.25,0.4,0.2,0.1,0.05)],
full_house = []
col_names = ['Tree_Name','n','Class','W1-K4-R','W2-K4-R','W3-K4-R']

result = total_df.groupby('Tree Name')['Class'].sum().reset_index()
trees = list(result['Tree Name'])
for tree in trees:
  try:

    df_chose = total_df[total_df.index == tree]
    order = list(df_chose['Comment_n'])[1:]
    cls =  list(df_chose['Class'])[1:]
    embeddings =  list(df_chose['Embeddings'])[1:]


    # process root
    root = df_chose[df_chose['Comment_n'] == -1].Embeddings
    root = root.reset_index()
    root_txt = list(root['Embeddings'])[0]


    for i in order:
      cls_curr = cls[i]
      curr = embeddings[i]
      root_score = calculate_semantic_cosine(root_txt,curr)

      if i == 0:
        total_score = root_score
        full_house.append((tree,i,cls_curr,total_score,total_score,total_score))

      elif i == 1:
        prev_txt = embeddings[0]
        prev_score = calculate_semantic_cosine(curr,prev_txt)
        total_score = 0.6 * root_score + 0.4 * prev_score
        full_house.append((tree,i,cls_curr,total_score,total_score,total_score))

      elif i == 2:
        prev_txt_0 = embeddings[0]
        prev_txt_1 = embeddings[1]
        prev_score_0 = calculate_semantic_cosine(curr,prev_txt_0)
        prev_score_1 = calculate_semantic_cosine(curr,prev_txt_1)
        total_score = 0.5 * root_score + 0.35 * prev_score_0 + 0.15 * prev_score_1
        full_house.append((tree,i,cls_curr,total_score,total_score,total_score))

      elif i == 3:
        prev_txt_0 = embeddings[0]
        prev_txt_1 = embeddings[1]
        prev_txt_2 = embeddings[2]

        prev_score_0 = calculate_semantic_cosine(curr,prev_txt_0)
        prev_score_1 = calculate_semantic_cosine(curr,prev_txt_1)
        prev_score_2 = calculate_semantic_cosine(curr,prev_txt_2)

        total_score = 0.5 * root_score + 0.35 * prev_score_0 + 0.15 * prev_score_1 + 0.10 * prev_score_2
        full_house.append((tree,i,cls_curr,total_score,total_score,total_score))

      elif i == 4:
        prev_txt_0 = embeddings[0]
        prev_txt_1 = embeddings[1]
        prev_txt_2 = embeddings[2]
        prev_txt_3 = embeddings[3]

        prev_score_0 = calculate_semantic_cosine(curr,prev_txt_0)
        prev_score_1 = calculate_semantic_cosine(curr,prev_txt_1)
        prev_score_2 = calculate_semantic_cosine(curr,prev_txt_2)
        prev_score_3 = calculate_semantic_cosine(curr,prev_txt_3)

        total_score = 0.4 * root_score + 0.30 * prev_score_0 + 0.15 * prev_score_1 + 0.10 * prev_score_2 + 0.05 * prev_score_3
        full_house.append((tree,i,cls_curr,total_score,total_score,total_score))

      else:
        diff_w = []
        for w_set in weights:
          w1,w2,w3,w4,w5 = w_set

          prev_txt_0 = embeddings[i-1]
          prev_txt_1 = embeddings[i-2]
          prev_txt_2 = embeddings[i-3]
          prev_txt_3 = embeddings[i-4]

          prev_score_0 = calculate_semantic_cosine(curr,prev_txt_0)
          prev_score_1 = calculate_semantic_cosine(curr,prev_txt_1)
          prev_score_2 = calculate_semantic_cosine(curr,prev_txt_2)
          prev_score_3 = calculate_semantic_cosine(curr,prev_txt_3)

          total_score = w1 * root_score + w2 * prev_score_0 + w3 * prev_score_1 + w4 * prev_score_2 + w5 * prev_score_3
          diff_w.append(total_score)

        full_house.append((tree,i,cls_curr,diff_w[0],diff_w[1],diff_w[2]))

  except:
    print('Error')

full_house_df = pd.DataFrame(full_house,columns = col_names)
all_df = all_df.merge(full_house_df.iloc[:, 3:], left_index=True, right_index=True)


## K Sliding Window Without root

### K == 1

In [ ]:
weights = [(0.5,0.5), (0.25,0.75), (0.75,0.25)]
root_w = -1
full_house = []
col_names = ['Tree_Name','n','Class','W1-K1-WR','W2-K1-WR','W3-K1-WR']


result = total_df.groupby('Tree Name')['Class'].sum().reset_index()
trees = list(result['Tree Name'])


for tree in trees:
  try:
    print(tree)
    df_chose = total_df[total_df.index == tree]
    order = list(df_chose['Comment_n'])[1:]
    cls =  list(df_chose['Class'])[1:]
    embeddings =  list(df_chose['Embeddings'])[1:]


    for i in order:
      cls_curr = cls[i]
      curr = embeddings[i]

      if i == 0:
        total_score = 0
        full_house.append((tree,i,cls_curr,total_score,total_score,total_score))

      else:
        diff_w = []
        for w_set in weights:
          w1,w2 = w_set
          w = w1 + w2
          prev_txt_0 = embeddings[i-1]
          prev_score_0 = calculate_semantic_cosine(curr,prev_txt_0)
          total_score = prev_score_0
          diff_w.append(total_score)

        full_house.append((tree,i,cls_curr,diff_w[0],diff_w[1],diff_w[2]))

  except:
    print('Error')

full_house_df = pd.DataFrame(full_house,columns = col_names)
all_df = all_df.merge(full_house_df.iloc[:, 3:], left_index=True, right_index=True)


### K == 2

In [ ]:
weights = [(0.5,0.5), (0.333, 0.666), (0.666,0.333)]
full_house = []
col_names = ['Tree_Name','n','Class','W1-K2-WR','W2-K2-WR','W3-K2-WR']

result = total_df.groupby('Tree Name')['Class'].sum().reset_index()
trees = list(result['Tree Name'])

for tree in trees:
  try:

    df_chose = total_df[total_df.index == tree]
    order = list(df_chose['Comment_n'])[1:]
    cls =  list(df_chose['Class'])[1:]
    embeddings =  list(df_chose['Embeddings'])[1:]

    for i in order:
      cls_curr = cls[i]
      curr = embeddings[i]

      if i == 0:
        total_score = 0
        full_house.append((tree,i,cls_curr,total_score,total_score,total_score))

      elif i == 1:
        prev_txt = embeddings[0]
        prev_score = calculate_semantic_cosine(curr,prev_txt)
        total_score = prev_score
        full_house.append((tree,i,cls_curr,total_score,total_score,total_score))

      else:
        diff_w = []
        for w_set in weights:
          w1,w2 = w_set

          prev_txt_0 = embeddings[i-1]
          prev_txt_1 = embeddings[i-2]

          prev_score_0 = calculate_semantic_cosine(curr,prev_txt_0)
          prev_score_1 = calculate_semantic_cosine(curr,prev_txt_1)
          total_score =  w1 * prev_score_0 + w2 * prev_score_1
          diff_w.append(total_score)

        full_house.append((tree,i,cls_curr,diff_w[0],diff_w[1],diff_w[2]))

  except:
    print('Error')

full_house_df = pd.DataFrame(full_house,columns = col_names)
all_df = all_df.merge(full_house_df.iloc[:, 3:], left_index=True, right_index=True)


### K == 3

In [ ]:
weights = [(0.333,0.333,0.333), (0.55,0.30,0.15), (0.65,0.20,0.15)],
full_house = []
col_names = ['Tree_Name','n','Class','W1-K3-WR','W2-K3-WR','W3-K3-WR']

result = total_df.groupby('Tree Name')['Class'].sum().reset_index()
trees = list(result['Tree Name'])

for tree in trees:
  try:

    df_chose = total_df[total_df.index == tree]
    order = list(df_chose['Comment_n'])[1:]
    cls =  list(df_chose['Class'])[1:]
    embeddings =  list(df_chose['Embeddings'])[1:]

    for i in order:
      cls_curr = cls[i]
      curr = embeddings[i]

      if i == 0:
        total_score = 0
        full_house.append((tree,i,cls_curr,total_score,total_score,total_score))

      elif i == 1:
        prev_txt = embeddings[0]
        prev_score = calculate_semantic_cosine(curr,prev_txt)
        total_score = prev_score
        full_house.append((tree,i,cls_curr,total_score,total_score,total_score))

      elif i == 2:
        prev_txt_0 = embeddings[0]
        prev_txt_1 = embeddings[1]

        prev_score_0 = calculate_semantic_cosine(curr,prev_txt_0)
        prev_score_1 = calculate_semantic_cosine(curr,prev_txt_1)

        total_score =  0.65 * prev_score_0 + 0.35 * prev_score_1

        full_house.append((tree,i,cls_curr,total_score,total_score,total_score))

      else:
        diff_w = []
        for w_set in weights:
          w1,w2,w3 = w_set

          prev_txt_0 = embeddings[i-1]
          prev_txt_1 = embeddings[i-2]
          prev_txt_2 = embeddings[i-3]

          prev_score_0 = calculate_semantic_cosine(curr,prev_txt_0)
          prev_score_1 = calculate_semantic_cosine(curr,prev_txt_1)
          prev_score_2 = calculate_semantic_cosine(curr,prev_txt_2)

          total_score = w1 * prev_score_0 + w2 * prev_score_1 + w3 * prev_score_2
          diff_w.append(total_score)

        full_house.append((tree,i,cls_curr,diff_w[0],diff_w[1],diff_w[2]))

  except:
    print('Error')

full_house_df = pd.DataFrame(full_house,columns = col_names)
all_df = all_df.merge(full_house_df.iloc[:, 3:], left_index=True, right_index=True)


### K == 4

In [ ]:
weights = [(0.25,0.25,0.25,0.25), (0.4,0.3,0.20,0.10), (0.5,0.25,0.15,0.10)],
full_house = []
col_names = ['Tree_Name','n','Class','W1-K4-WR','W2-K4-WR','W3-K4-WR']

result = total_df.groupby('Tree Name')['Class'].sum().reset_index()
trees = list(result['Tree Name'])
for tree in trees:
  try:

    df_chose = total_df[total_df.index == tree]
    order = list(df_chose['Comment_n'])[1:]
    cls =  list(df_chose['Class'])[1:]
    embeddings =  list(df_chose['Embeddings'])[1:]

    for i in order:
      cls_curr = cls[i]
      curr = embeddings[i]
      root_score = calculate_semantic_cosine(root_txt,curr)

      if i == 0:
        total_score = 0
        full_house.append((tree,i,cls_curr,total_score,total_score,total_score))

      elif i == 1:
        prev_txt = embeddings[0]
        prev_score = calculate_semantic_cosine(curr,prev_txt)
        total_score = prev_score
        full_house.append((tree,i,cls_curr,total_score,total_score,total_score))

      elif i == 2:
        prev_txt_0 = embeddings[0]
        prev_txt_1 = embeddings[1]

        prev_score_0 = calculate_semantic_cosine(curr,prev_txt_0)
        prev_score_1 = calculate_semantic_cosine(curr,prev_txt_1)

        total_score = 0.6 * prev_score_0 + 0.4 * prev_score_1
        full_house.append((tree,i,cls_curr,total_score,total_score,total_score))

      elif i == 3:
        prev_txt_0 = embeddings[0]
        prev_txt_1 = embeddings[1]
        prev_txt_2 = embeddings[2]

        prev_score_0 = calculate_semantic_cosine(curr,prev_txt_0)
        prev_score_1 = calculate_semantic_cosine(curr,prev_txt_1)
        prev_score_2 = calculate_semantic_cosine(curr,prev_txt_2)

        total_score = 0.5 * prev_score_0 + 0.35 * prev_score_1 + 0.15 * prev_score_2
        full_house.append((tree,i,cls_curr,total_score,total_score,total_score))

      else:
        diff_w = []
        for w_set in weights:
          w1,w2,w3,w4 = w_set

          prev_txt_0 = embeddings[i-1]
          prev_txt_1 = embeddings[i-2]
          prev_txt_2 = embeddings[i-3]
          prev_txt_3 = embeddings[i-4]

          prev_score_0 = calculate_semantic_cosine(curr,prev_txt_0)
          prev_score_1 = calculate_semantic_cosine(curr,prev_txt_1)
          prev_score_2 = calculate_semantic_cosine(curr,prev_txt_2)
          prev_score_3 = calculate_semantic_cosine(curr,prev_txt_3)

          total_score = w1 * prev_score_0 + w2 * prev_score_1 + w3 * prev_score_2 + w4 * prev_score_3
          diff_w.append(total_score)

        full_house.append((tree,i,cls_curr,diff_w[0],diff_w[1],diff_w[2]))

  except:
    print('Error')

full_house_df = pd.DataFrame(full_house,columns = col_names)
all_df = all_df.merge(full_house_df.iloc[:, 3:], left_index=True, right_index=True)
all_df.to_csv('Final.csv')

## Length and readability score features

In [ ]:
full_house = []
for tree in trees:
  try:
    df_chose = df[df['tree_id'] == tree]
    df_chose = df_chose[df_chose['text'].apply(lambda x: len(x.split()) >= 7)]
    # create groups
    all_txt = list(df_chose['text'])[1:]
    count = 0
    for text in all_txt:
      l = len(text)
      read_score = calculate_readability_score(text)
      full_house.append((tree,count,l,read_score))
      count += 1


  except Exception as e:
    print(f'Error in {tree} - {e}')

In [ ]:
# full_house_df = pd.DataFrame(full_house,columns = ['Tree Name','i','Class','W1','W2','W3'])
# full_house_df = pd.DataFrame(full_house,columns = ['Tree Name','i','Length','Read_score'])
# full_house_df.to_csv('Full_house_more.csv')

## Classification with ML

In [ ]:
df = pd.read_csv('all.csv')
df = df.iloc[:,3:]

In [ ]:
X = df.iloc[:,1:].values
y = df.iloc[:,0].values

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

from xgboost import XGBClassifier


# Split the data into train and test sets (stratified)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

# Use RandomUnderSampler to perform undersampling on the majority class
# rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)
# X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

# Use RandomOverSampler to perform oversampling on the minority class
ros = RandomOverSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
# Initialize classifiers
classifiers = {
    'XGBoost': XGBClassifier(),
    'Linear SVM': SVC(kernel='rbf'),
    'Logistic Regression': LogisticRegression(penalty='none'),
    'KNN': KNeighborsClassifier(n_neighbors=11)
}

# Train and evaluate each classifier
for name, clf in classifiers.items():
    print(f"Training {name}...")
    clf.fit(X_resampled, y_resampled)

    # Make predictions on the test set
    y_pred = clf.predict(X_test)

    # Display classification report
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    # Display confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for {name}:")
    print(conf_matrix)
    print("\n" + "-"*50 + "\n")


Training XGBoost...
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.95      0.94      2190
           1       0.05      0.04      0.05       141

    accuracy                           0.89      2331
   macro avg       0.49      0.50      0.49      2331
weighted avg       0.89      0.89      0.89      2331

Confusion Matrix for XGBoost:
[[2076  114]
 [ 135    6]]

--------------------------------------------------

Training Linear SVM...
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.41      0.57      2190
           1       0.07      0.65      0.12       141

    accuracy                           0.42      2331
   macro avg       0.51      0.53      0.34      2331
weighted avg       0.89      0.42      0.54      2331

Confusion Matrix for Linear SVM:
[[ 892 1298]
 [  49   92]]

--------------------------------------------------

Training Logistic Regression...
Cl

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.68      0.79      2190
           1       0.07      0.38      0.12       141

    accuracy                           0.66      2331
   macro avg       0.51      0.53      0.46      2331
weighted avg       0.89      0.66      0.75      2331

Confusion Matrix for KNN:
[[1492  698]
 [  87   54]]

--------------------------------------------------



## Classficiation with Thresholds

In [ ]:
df = pd.read_csv('all.csv')
df = df.iloc[:,1:-5]
df

,Tree Name,i,Class,W1_c,W2_c,W3_c
0,4r2a4d,0,0,0.315199,0.315199,0.315199
1,4r2a4d,1,0,0.416977,0.416977,0.416977
2,4r2a4d,2,0,0.395174,0.395174,0.395174
3,4r2a4d,3,0,0.371214,0.466018,0.432945
4,4r2a4d,4,0,0.245730,0.274229,0.260205
...,...,...,...,...,...,...
9316,7yf2le,39,0,0.590449,0.600688,0.585739
9317,7yf2le,40,0,0.513397,0.568745,0.553209
9318,7yf2le,41,0,0.253096,0.255947,0.250931
9319,7yf2le,42,1,0.121275,0.081300,0.088650


In [ ]:
thresholds = [0.35, 0.4, 0.45, 0.5, 0.55, 0.60, 0.65, 0.7]

# Threshold function
def apply_threshold(value, threshold):
    return 1 if value >= threshold else 0

w1_c = list(df['W1_c'])
w2_c = list(df['W2_c'])
w3_c = list(df['W3_c'])

cols = ['W1_c',	'W2_c',	'W3_c']
for threshold_value in thresholds:
  for col in cols:
      df[f'{col} - {threshold_value}'] = df[col].apply(apply_threshold, threshold=threshold_value)

In [ ]:
y_true = df.iloc[:,2].values

y_preds = []
cols_to_calc = df.columns[6:]
for col in cols_to_calc:
  y_preds.append(list(df[col]))


In [ ]:
count = 0
for y_pred in y_preds:
     # Display classification report
    print(f'***********************************************   {cols_to_calc[count]}   ***********************************************')
    print("Classification Report:")
    print(classification_report(y_true, y_pred))

    # Display confusion matrix
    conf_matrix = confusion_matrix(y_true, y_pred)
    print(f"Confusion Matrix")
    print(conf_matrix)
    print("\n" + "-"*50 + "\n")
    count += 1

***********************************************   W1_c - 0.35   ***********************************************
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.25      0.39      8756
           1       0.07      0.82      0.12       565

    accuracy                           0.28      9321
   macro avg       0.51      0.53      0.26      9321
weighted avg       0.90      0.28      0.38      9321

Confusion Matrix
[[2179 6577]
 [ 104  461]]

--------------------------------------------------

***********************************************   W2_c - 0.35   ***********************************************
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.23      0.37      8756
           1       0.07      0.84      0.12       565

    accuracy                           0.27      9321
   macro avg       0.51      0.53      0.25      9321
weighted avg       0.90      0.27 